In [1]:
import cv2
import numpy as np
import os
import mediapipe as mp

from closet_module import capture_module as cm
from closet_module import pose_module as pm
from closet_module import synthesis_module as sm
from closet_module import yolo_module as ym
from closet_module import cut_img_module as cim
from closet_module import grab_cut_module as gm
from closet_module import resize as re

In [2]:
#아바타 선택 함수
def choice_avatar():
    path = 'C:\\Users\\lky\\DeepLearning\\smart_closet\\img_lib\Avatar\\'
    file_list = os.listdir(path)
    
    img_list=[]
    for i in range (len(file_list)):
        img=cv2.imread(path+f'Avatar{i+1}.png')
        img_list.append(img)
        cv2.imshow(f'Avatar{i+1}',img_list[i])
    
    cv2.waitKey()
    cv2.destroyAllWindows()
    
    num = input('원하는 아바타를 선택하여 주십시요:')
    avatar = path + f'Avatar{int(num)}.png'
    
    return avatar

In [3]:
#옷 카테고리 선택 함수
def choice_category():
    classes=['short sleeve top','long sleeve top','short sleeve outwear','long sleeve outwear','vest','sling','shorts','trousers','skirt','short sleeve dress','long sleeve dress','vest dress','sling dress']
    print('0:short sleeve top\n1:long sleeve top\n2:short sleeve outwear\n3:long sleeve outwear\n4:vest\n5:sling\n6:shorts\n7:trousers\n8:skirt\n9:short sleeve dress\n10:long sleeve dress\n11:vest dress\n12:sling dress')
    num = input('옷의 종류를 선택해 주십시요:')
    i = int(num)
    label = classes[i]
    
    return label

In [4]:
def choice_cloth(label):
    cloth_list=[]
    path='C:\\Users\\lky\\DeepLearning\\smart_closet\\img_lib\\cut_img\\'+label
    
    cloth_list = os.listdir(path)
    
    print(cloth_list)
        
    img = input('옷을 선택하여 주십시요:')
    cloth = path+'\\'+img
    
    return cloth

In [5]:
##################################
#        아바타 정하기           #
##################################
avatar_path=choice_avatar()
print(avatar_path)

원하는 아바타를 선택하여 주십시요:1
C:\Users\lky\DeepLearning\smart_closet\img_lib\Avatar\Avatar1.png


In [5]:
##############################
#     아바타 캡쳐 과정       #
##############################
cm.cap()

q=캡쳐 후 종료,w=종료


In [6]:
####################################
#          옷 추출 과정            #
####################################
#포즈로 웹캠 캡쳐하기
pm.check_image_angle(avatar_path)
cap_img=pm.video_angle()

In [7]:
#옷 detecting 과정
cloth_det=ym.detect_cloth(cap_img)

원하는 옷의 종류를 선택하여 주십시요:
0번:long sleeve top 
0


In [8]:
#grab_cut알고리즘으로 옷 추출
grab_img = gm.grabcut(cap_img,cloth_det[0],cloth_det[1],cloth_det[2],cloth_det[3])
avatar_img=cv2.imread(avatar_path)

오른쪽 마우스 버튼을 누르고 영역을 지정한 후 n을 누르세요
n:적용하기
0:제거배경선택 1:복원전경선택 n:적용하기 r:리셋 s:저장 q:종료
왼쪽 마우스로 제거할 부분을 표시한 후 n을 누르세요
0:제거배경선택 1:복원전경선택 n:적용하기 r:리셋 s:저장 q:종료
0:제거배경선택 1:복원전경선택 n:적용하기 r:리셋 s:저장 q:종료
0:제거배경선택 1:복원전경선택 n:적용하기 r:리셋 s:저장 q:종료
0:제거배경선택 1:복원전경선택 n:적용하기 r:리셋 s:저장 q:종료
0:제거배경선택 1:복원전경선택 n:적용하기 r:리셋 s:저장 q:종료
0:제거배경선택 1:복원전경선택 n:적용하기 r:리셋 s:저장 q:종료
왼족 마우스로 복원할 부분을 표시한 후 n을 누르세요
0:제거배경선택 1:복원전경선택 n:적용하기 r:리셋 s:저장 q:종료
0:제거배경선택 1:복원전경선택 n:적용하기 r:리셋 s:저장 q:종료
0:제거배경선택 1: 복원전경선택 n:적용하기 r:리셋 s:저장 q:종료


In [9]:
print('1:top, 2:bottom')
a=input('원하는 옷의 위치를 선택하여 주세요:')
if a == '1':
    choice='top'
elif a == '2':
    choice='bottom'
    
resize=re.resize(avatar_img,cap_img,grab_img,choice)
cut_img=resize.cut()

#추출한 옷 저장
save_path='C:\\Users\\lky\\DeepLearning\\smart_closet\\img_lib\\cut_img\\'

i=1
while(True):
    file = save_path+resize.cloth_position[4]+'\\'+resize.cloth_position[4]+f'{i}'+'.png'
    if os.path.isfile(file):
        i+=1
        continue
    cv2.imwrite(save_path+resize.cloth_position[4]+'\\'+resize.cloth_position[4]+f'{i}'+'.png',cut_img)
    break

1:top, 2:bottom
원하는 옷의 위치를 선택하여 주세요:1
원하는 옷의 종류를 선택하여 주십시요:
0번:short sleeve top 1번:short sleeve top 2번:trousers 3번:trousers 
0
원하는 옷의 종류를 선택하여 주십시요:
0번:long sleeve top 
0


In [10]:
####################################################
#                   옷 피팅 과정                   #
####################################################
path1=choice_avatar()
avatar=cv2.imread(path1)
avatar_position=ym.detect_cloth(avatar)
while True:
    k =input('n:합성, q:종료')
    
    if k == 'n':
        label=choice_category()
        cloth_path=choice_cloth(label)
        
        #합성할 이미지 경로
        cap_img=cv2.imread(cloth_path)

        #이미지 합성
        syn_img = sm.synthesis(avatar_position[0],avatar_position[1],avatar,cloth_path)
        print('n:합성 ,q:종료')
        avatar=syn_img
    
    elif k == 'q':
        break
        
cv2.imshow('img',avatar)
cv2.waitKey()
cv2.destroyAllWindows()
        

원하는 아바타를 선택하여 주십시요:1
원하는 옷의 종류를 선택하여 주십시요:
0번:short sleeve top 1번:short sleeve top 2번:trousers 3번:trousers 
0
n:합성, q:종료n
0:short sleeve top
1:long sleeve top
2:short sleeve outwear
3:long sleeve outwear
4:vest
5:sling
6:shorts
7:trousers
8:skirt
9:short sleeve dress
10:long sleeve dress
11:vest dress
12:sling dress
옷의 종류를 선택해 주십시요:1
['long sleeve top1.png', 'long sleeve top2.png', 'long sleeve top3.png', 'long sleeve top4.png', 'long sleeve top5.png', 'long sleeve top6.png']
옷을 선택하여 주십시요:long sleeve top6.png
n:합성 ,q:종료
n:합성, q:종료q
